In [2]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def find_loops(adj_matrix):
    """
    使用深度优先搜索 (DFS) 找到所有独立回路
    """
    n = adj_matrix.shape[0]
    visited = [False] * n
    loops = []
    
    def dfs(node, path):
        if visited[node]:
            # 如果当前节点已经被访问过，说明找到了一个回路
            loop_start = path.index(node)
            loops.append(path[loop_start:])
            return
        visited[node] = True
        for neighbor in range(n):
            if adj_matrix[node, neighbor] != 0:
                dfs(neighbor, path + [neighbor])
        visited[node] = False
    
    for i in range(n):
        if not visited[i]:
            dfs(i, [i])
    
    # 去除重复的回路
    unique_loops = []
    for loop in loops:
        if len(loop) > 2 and loop not in unique_loops:
            unique_loops.append(loop)
    
    return unique_loops

def generate_node_equations(adj_matrix, flows, node_demands, node_pressures, pipe_index):
    """
    生成节点流量守恒方程，包括节点的外部需求
    """
    n = adj_matrix.shape[0]
    node_eqs = []
    
    for i in range(n):
        # 计算流入和流出
        incoming = 0
        outgoing = 0
        for j in range(n):
            if adj_matrix[j, i] < 0:  # 流入
                incoming += flows[j, i]
            elif adj_matrix[i, j] < 0:  # 流出
                outgoing -= flows[i, j]
        
        # 节点的流量守恒方程：流入 - 流出 + 需求 = 0
        node_eq = incoming - outgoing + node_demands[i]
        node_eqs.append(node_eq)
    
    return np.array(node_eqs)

def generate_loop_equations(loops, adj_matrix, flows, resistances, pipe_index):
    """
    生成回路压降守恒方程
    """
    loop_eqs = []
    
    for loop in loops:
        loop_eq = 0
        for i in range(len(loop)):
            start = loop[i]
            end = loop[(i + 1) % len(loop)]
            if (start, end) in pipe_index:  # 检查边是否存在
                idx = pipe_index[(start, end)]
                flow = flows[start, end]
                resistance = resistances[start, end]
                loop_eq += flow * resistance
        loop_eqs.append(loop_eq)
    
    return np.array(loop_eqs)

def solve_network(adj_matrix, flows, resistances, node_demands, node_pressures):
    """
    解决管网系统的平衡方程，包括节点的外部需求和指定的节点压力
    """
    n = adj_matrix.shape[0]
    
    # 确定管道数量
    num_pipes = np.sum(np.abs(adj_matrix) > 0)
    
    # 映射管道索引
    pipe_index = {}
    count = 0
    for i in range(n):
        for j in range(n):
            if adj_matrix[i, j] != 0:
                pipe_index[(i, j)] = count
                count += 1
    
    # 生成节点方程
    node_eqs = generate_node_equations(adj_matrix, flows, node_demands, node_pressures, pipe_index)
    
    # 生成回路方程
    loops = find_loops(adj_matrix)
    loop_eqs = generate_loop_equations(loops, adj_matrix, flows, resistances, pipe_index)
    
    # 组合方程
    num_known_pressures = np.count_nonzero(node_pressures)
    num_unknown_pressures = n - num_known_pressures
    num_equations = num_unknown_pressures +num_known_pressures #+ len(loops)
    
    if num_equations != num_pipes:
        raise ValueError(f"Number of equations ({num_equations}) does not match number of pipes ({num_pipes})")
    
    A = np.zeros((num_equations, num_pipes))

    print(A)
    b = np.zeros(num_pipes) #+len(num_pipes)) num_equations+

    b = np.full_like(b, 0.1, dtype=float) 
    print(b)
    
    # 填充节点方程
    row_idx = 0
    for i in range(n):
        if node_pressures[i] == 0:  # 只处理未知压力的节点
            for j in range(n):
                if adj_matrix[j, i] < 0:  # 流入
                    idx = pipe_index[(j, i)]
                    A[row_idx, idx] = 1
                elif adj_matrix[i, j] < 0:  # 流出
                    idx = pipe_index[(i, j)]
                    A[row_idx, idx] = -1
            b[row_idx] = -node_eqs[i]
            row_idx += 1
        else:  # 已知压力的节点
            b[row_idx] = node_pressures[i]
            row_idx += 1
    
    # 填充回路方程
    for i, loop in enumerate(loops):
        for j in range(len(loop)):
            start = loop[j]
            end = loop[(j + 1) % len(loop)]
            if (start, end) in pipe_index:  # 检查边是否存在
                idx = pipe_index[(start, end)]
                A[row_idx, idx] = resistances[start, end]
        b[row_idx] = -loop_eqs[i]
        row_idx += 1
    
    # 求解线性方程组
    x = spsolve(csr_matrix(A), b)
    
    return x

# 示例数据
adj_matrix = np.array([
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, -1],
    [1, 0, 0, 0]
])

# 初始化流量分布为非零值
flows = np.full_like(adj_matrix, 0.1, dtype=float)  # 初始流量分布，每个管道有 0.1 的流量

resistances = np.array([
    [0, 1, 0, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 3],
    [4, 0, 0, 0]
])

# 增加节点的外部需求
node_demands = np.array([0, 5, -3, 2])  # 示例需求，单位可以是立方米/秒等

# 指定的节点压力
node_pressures = np.array([10, 0, 0, 0])  # 假设第一个节点的压力是已知的，其他节点的压力未知

# 求解
solution = solve_network(adj_matrix, flows, resistances, node_demands, node_pressures)
print("Solution:", solution)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[0.1 0.1 0.1 0.1]


IndexError: index 4 is out of bounds for axis 0 with size 4